In [3]:
%load_ext raw_magic

# Drill down XML

The XML file `patients.xml` is an export from a Hospital database with (simulated) data from 100 patients. The XML contains multiple levels of hierarchies with information on medication, forms, hospital stays, etc.

In [4]:
%buckets_register raw-tutorial

API error: S3 credentials already exists


In [5]:
%%query
read_xml("s3://raw-tutorial/ipython-demos/patients.xml")

RAW's SQL extensions allow us to "drill down" the hierarchy and extract a report on nursing times per patient without explicitly flatening the XML file in advance.

In [6]:
%%query

patients := read_xml("s3://raw-tutorial/ipython-demos/patients.xml");

SELECT surname, SUM(totalMinutes) AS nurseMinutes
FROM patients.item, chartBasedData.nursingReports.item
GROUP BY name, surname

Showing only 100 values...


surname,nurseMinutes
Stewart,20
Butler,45
Moore,10
Walker,25
Nelson,50
Long,40
Baker,15
Adams,10
Parker,15
Kelly,40


# Creating a custom record

In [11]:
%%query
(
    text: "This is a custom record created using raw",
    year: 2020,
    list: select * from [1, 2, 3, 4]    
)

# From CSV to hierarchical data

The file `diagnosis.csv` contains a flat table with diagnosis. 

In [5]:
%%query
read("s3://raw-tutorial/ipython-demos/diagnosis.csv")

Showing only 100 values...


patient_id,code,description,sub_category,category,diag_date
P00022,L29.3,"Anogenital pruritus, unspecified",Dermatitis and eczema,Diseases of the skin and subcutaneous tissue,2000/02/22
P00083,L81.1,Chloasma,Other disorders of the skin and subcutaneous tissue,Diseases of the skin and subcutaneous tissue,2000/02/25
P00017,L11.0,Acquired keratosis follicularis,Bullous disorders,Diseases of the skin and subcutaneous tissue,2000/03/01
P00026,L28.1,Prurigo nodularis,Dermatitis and eczema,Diseases of the skin and subcutaneous tissue,2000/03/19
P00072,L03.3,Cellulitis of trunk,Infections of the skin and subcutaneous tissue,Diseases of the skin and subcutaneous tissue,2000/04/17
P00068,L71.1,Rhinophyma,Disorders of skin appendages,Diseases of the skin and subcutaneous tissue,2000/06/17
P00032,L60.0,Ingrowing nail,Disorders of skin appendages,Diseases of the skin and subcutaneous tissue,2000/06/23
P00068,L70.8,Other acne,Disorders of skin appendages,Diseases of the skin and subcutaneous tissue,2000/07/05
P00083,L25.2,Unspecified contact dermatitis due to dyes,Dermatitis and eczema,Diseases of the skin and subcutaneous tissue,2000/09/11
P00085,L90.4,Acrodermatitis chronica atrophicans,Other disorders of the skin and subcutaneous tissue,Diseases of the skin and subcutaneous tissue,2000/11/08


The following query converts the flat table into a hierarchical structure, where each row is now a nested record with a category of diagnosis and a list descriptions.

**Note that * in a GROUP BY query refers to the elements in each group.**

In [12]:
%%query
diagnosis := read("s3://raw-tutorial/ipython-demos/diagnosis.csv");

SELECT category,
       (SELECT COUNT(patient_id) as N, description FROM * GROUP BY  description) AS descriptions
FROM diagnosis
GROUP BY category


# Join XML and CSV

This query joins the CSV and XML files, extracting patient ages to generate a distribution for a certain type of disease.

**Note that no schemas were created, no data was explicitly loaded and no separate ETL process or scripts were needed: these optimizations are all done internally by RAW and transparent to the user.**

In [7]:
%%query

patients := read_xml("s3://raw-tutorial/ipython-demos/patients.xml");

ages := SELECT patient_id,
               2019 - YEAR(TO_DATE(birthDate, "yyyy/MM/dd")) AS age,
               gender
        FROM patients.item;

diagnosis := read("s3://raw-tutorial/ipython-demos/diagnosis.csv");

SELECT age_group,
       CCOUNT(SELECT gender FROM * WHERE gender="female") AS female,
       CCOUNT(SELECT gender FROM * WHERE gender="male") AS male
FROM ages a, diagnosis d
WHERE a.patient_id = d.patient_id
  AND d.category = "Diseases of the skin and subcutaneous tissue"
GROUP BY 10*(a.age/10) AS age_group

age_group,female,male
0,6,1
10,9,14
20,16,17
30,25,17
40,14,18
50,24,0
60,0,8
70,0,3
80,0,2
90,4,3
